In [84]:
import yfinance as yff
import pandas as pd
start_date = '2024-01-01'

In [85]:
tickers = yf.download(['TSLA', 'GME', 'AAPL', 'NVDA'], start=start_date)
tickers = tickers.dropna()
tickers.tail(100)

[*********************100%***********************]  4 of 4 completed


Price            Close                                       High             \
Ticker            AAPL    GME        NVDA        TSLA        AAPL        GME   
Date                                                                           
2024-11-04  221.522446  22.33  136.028122  242.839996  222.300732  22.650000   
2024-11-05  222.959290  22.99  139.887497  251.440002  223.458192  23.200001   
...                ...    ...         ...         ...         ...        ...   
2025-03-28  217.899994  21.73  109.669998  263.549988  223.809998  22.790001   
2025-03-31  220.884995  21.93  105.940002  254.800003  220.929993  21.990000   

Price                                      Low                         \
Ticker            NVDA        TSLA        AAPL        GME        NVDA   
Date                                                                    
2024-11-04  138.937658  248.899994  219.227509  21.879999  135.548203   
2024-11-05  140.347414  255.279999  220.654365  22.270000  137.307910   
...                ...         ...         ...        ...         ...   
2025-03-28  112.870003  276.100006  217.679993  21.700001  109.070000   
2025-03-31  106.010002  255.250000  216.240005  20.730101  103.650002   

Price                         Open                                     \
Ticker            TSLA        AAPL        GME        NVDA        TSLA   
Date                                                                    
2024-11-04  238.880005  220.504697  22.200001  137.187939  244.559998   
2024-11-05  246.210007  221.312919  22.320000  137.427886  247.339996   
...                ...         ...        ...         ...         ...   
2025-03-28  260.570007  221.669998  21.889999  111.489998  275.579987   
2025-03-31  243.360107  217.220001  21.400000  105.110001  249.169998   

Price         Volume                                  
Ticker          AAPL       GME       NVDA       TSLA  
Date                                                  
2024-11-04  44944500   4300200  187528200   68802400  
2024-11-05  28111300   5858400  160537400   69282500  
...              ...       ...        ...        ...  
2025-03-28  39784100  44700000  229177400  123226900  
2025-03-31  22461790   8417361  151978892   75045938  

[100 rows x 20 columns]

In [ ]:
# Create DataFrames for the new indicators with multi-level columns
indicators_columns = pd.MultiIndex.from_product([
    ['Daily_Return', 'MA5', 'MA20'],  # These will be new categories
    ticker_symbols
], names=['Price', 'Ticker'])

# Create empty DataFrame with the right structure
indicators = pd.DataFrame(index=tickers.index, columns=indicators_columns)

# Calculate indicators for each ticker
for ticker in ticker_symbols:
    # Daily Returns
    indicators[('Daily_Return', ticker)] = tickers[('Close', ticker)].pct_change()
    
    # 5-day Moving Average
    indicators[('MA5', ticker)] = tickers[('Close', ticker)].rolling(window=5).mean()
    
    # 20-day Moving Average
    indicators[('MA20', ticker)] = tickers[('Close', ticker)].rolling(window=20).mean()

# Concatenate with original DataFrame along columns axis
result = pd.concat([tickers, indicators], axis=1)

# Display the result
print(result.head())

# Save to CSV
result.to_csv('/Users/jonathanermias/Documents/GitHub/RedditStocks_v2.0/data/stock_data.csv')

Price            Close                                          High  \
Ticker            AAPL        GME       NVDA        TSLA        AAPL   
Date                                                                   
2024-01-02  184.532074  16.670000  48.149918  248.419998  187.315366   
2024-01-03  183.150391  16.690001  47.551140  238.449997  184.770668   
2024-01-04  180.824371  16.360001  47.979984  237.929993  181.997322   
2024-01-05  180.098694  15.960000  49.078568  237.490005  181.669266   
2024-01-08  184.452560  15.790000  52.233379  240.449997  184.492330   

Price                                                Low             \
Ticker            GME       NVDA        TSLA        AAPL        GME   
Date                                                                  
2024-01-02  17.600000  49.276493  251.250000  182.792518  16.580000   
2024-01-03  16.830000  48.165907  245.679993  182.335277  15.900000   
2024-01-04  16.750000  48.481795  242.699997  179.800520  16.309999 